<a href="https://colab.research.google.com/github/pam-lab/JupyterFiles/blob/main/Similarity_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Loading and processing

In [ ]:
!git clone https://github.com/language-ml/course-nlp-ir-1-text-exploring

fatal: destination path 'course-nlp-ir-1-text-exploring' already exists and is not an empty directory.


In [ ]:
import _pickle as pickle

class RomanizeUtility():

    def __init__(self,base_path):
        self.arabic2roman = RomanizeUtility.load_obj(f'{base_path}/data_config/arabic2roman.pickle')
        self.roman2arabic = RomanizeUtility.load_obj(f'{base_path}/data_config/roman2arabic.pickle')

    @staticmethod
    def load_obj(filename):
        return pickle.load(open(filename, "rb"))
        
    def get_arabic_token(self, token):
        '''
        :param token:
        :return:
        '''
        return ''.join([self.roman2arabic[e1] for e1 in list(token) if e1 in self.roman2arabic])

    def get_romanize_token(self, token):
        '''
        :param token:
        :return:
        '''
        return ''.join([self.arabic2roman[e1] for e1 in list(token) if e1 in self.arabic2roman])

    def get_arabic_text(self, text):
        '''
        :param text:
        :return:
        '''
        return  ' '.join([self.get_arabic_token(x) for x in text.split()])

In [ ]:
import random 
import nltk
import pandas as pd
import codecs
import tqdm

quranic_dir = "/content/course-nlp-ir-1-text-exploring/exploring-datasets/religious_text/quranic_data"
df = pd.read_csv(f'{quranic_dir}/id_text_with_orthographies.txt', sep='\t', header=None)
verse_complete_dict = pd.Series(df[1].tolist(), index=df[0]).to_dict()


# ROOT/LEMMA
ru=RomanizeUtility(quranic_dir)
verse_lemma_dict = {line.split()[1]:ru.get_arabic_text(' '.join(line.split()[2::]))
                     for line in [l.strip() 
                     for l in codecs.open(F'{quranic_dir}/id_quran_lem.txt','r','utf-8').readlines()]}
verse_root_dict = {line.split()[1]:ru.get_arabic_text(' '.join(line.split()[2::])) 
                    for line in [l.strip() 
                    for l in codecs.open(F'{quranic_dir}/id_quran_roots.txt','r','utf-8').readlines()]}

In [ ]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

In [ ]:
verse_lemma_dict['2##186']

'إِذَا سَأَلَ عَبْد عَن إِنّ قَرِیب أُجِیبَت دَعْوَة دَّاع إِذَا دَعَا ٱسْتَجَابَ ءَامَنَ لَعَلّ یَرْشُدُ'

In [ ]:
verse_root_dict['2##186']

'سال عبد قرب جوب دعو دعو دعو جوب امن رشد'

### Normalize

In [ ]:
pip install -Uq camel_tools

     |████████████████████████████████| 104 kB 4.4 MB/s 
     |████████████████████████████████| 175 kB 30.9 MB/s 
     |████████████████████████████████| 418 kB 40.3 MB/s 
     |████████████████████████████████| 4.2 MB 34.6 MB/s 
     |████████████████████████████████| 596 kB 40.7 MB/s 
     |████████████████████████████████| 6.6 MB 16.2 MB/s 
     |████████████████████████████████| 86 kB 4.8 MB/s 


In [ ]:
from camel_tools.utils.normalize import normalize_unicode
from camel_tools.utils.normalize import normalize_alef_maksura_ar
from camel_tools.utils.normalize import normalize_alef_ar

from camel_tools.utils.normalize import normalize_alef_bw
from camel_tools.utils.normalize import normalize_alef_hsb

from camel_tools.utils.normalize import normalize_teh_marbuta_ar
from camel_tools.utils.dediac import dediac_ar

def normalize_arabic(sentence):

    # Normalize alef variants to 'ا'
    sent_norm = normalize_unicode(sentence)
    
    sent_norm = normalize_alef_bw(sent_norm)
    # Normalize alef variants to 'ا'
    sent_norm = normalize_alef_ar(sentence)

    # Normalize alef maksura 'ى' to yeh 'ي'
    sent_norm = normalize_alef_maksura_ar(sent_norm)

    # Normalize teh marbuta 'ة' to heh 'ه'
    sent_norm = normalize_teh_marbuta_ar(sent_norm)
    return dediac_ar(sent_norm)

In [ ]:
from camel_tools.utils import normalize


verse_complete_dict_nrmlz = {k:normalize_arabic(v) for k,v in tqdm.tqdm(verse_complete_dict.items())}

100%|██████████| 6236/6236 [00:00<00:00, 10624.15it/s]


In [ ]:
verse_complete_dict['2##186']

'وَإِذَا سَأَلَكَ عِبَادِي عَنِّي فَإِنِّي قَرِيبٌ ۖ أُجِيبُ دَعْوَةَ الدَّاعِ إِذَا دَعَانِ ۖ فَلْيَسْتَجِيبُوا لِي وَلْيُؤْمِنُوا بِي لَعَلَّهُمْ يَرْشُدُونَ'

In [ ]:
verse_complete_dict_nrmlz['2##186']

'واذا سالك عبادي عني فاني قريب ۖ اجيب دعوه الداع اذا دعان ۖ فليستجيبوا لي وليؤمنوا بي لعلهم يرشدون'

In [ ]:
quranic_tokenized = [sents.split() for sents in tqdm.tqdm(verse_complete_dict_nrmlz.values())]

100%|██████████| 6236/6236 [00:00<00:00, 86210.84it/s]


In [ ]:
quranic_tokenized[43]

['فتلقي',
 'ادم',
 'من',
 'ربه',
 'كلمات',
 'فتاب',
 'عليه',
 'ۚ',
 'انه',
 'هو',
 'التواب',
 'الرحيم']